In [2]:
luci_mask = "./conf/masks/lhires-2400l-1785mm.mdf"
luci1 = "./conf/instruments/lhires-III-Halpha.icf"
luci_file = "./data-reduced/lamp-180s-001.fits"

In [23]:
from spectrapy.modelscalib.calib import ModelsCalibration

calib = ModelsCalibration(luci1, mask=luci_mask)



INFO: Loaded instrument configuration from ./conf/instruments/lhires-III-Halpha.icf [spectrapy.models.instrument]
INFO: Loaded mask from ./conf/masks/lhires-2400l-1785mm.mdf [spectrapy.mask.mask]


In [24]:
calib.load_image(luci_file)

opt = calib.new_opt_model(1, 1)

calib.plot_opt_model(edit=True)


INFO: Optical model center: 765.00, 510.00 [spectrapy.models.optmodel]
INFO: Optical model scales: x 346.26, y 96.95 [spectrapy.models.optmodel]


In [25]:
calib.fit_opt_model()
opt.writeto('examples/tmp/lhires.opt', overwrite=True)

INFO: Fitting mask [mm] positions vs region [pixel] positions [spectrapy.modelscalib.optcalib]
INFO: X model coeffs: chi2=inf R2=1 [spectrapy.models.optmodel]
INFO: Y model coeffs: chi2=inf R2=1 [spectrapy.models.optmodel]
INFO: Optical Model updated [spectrapy.models.optmodel]


/home/brezi-spec/anaconda3/envs/brezi-env/lib/python3.10/site-packages/spectrapy-1.0.1-py3.10-linux-x86_64.egg/spectrapy/math/poly.py:131: RuntimeWarning: divide by zero encountered in double_scalars
  chi2 = resids / (len(X) - nparam)


curvature model calibration

In [5]:
opt = './examples/tmp/lhires.opt'
from spectrapy.modelscalib.calib import ModelsCalibration
calib = ModelsCalibration(luci1, mask=luci_mask, opt=opt)
crv = calib.new_crv_model(2,0,1)

INFO: Loaded instrument configuration from ./conf/instruments/lhires-III-Halpha.icf [spectrapy.models.instrument]
INFO: Loaded mask from ./conf/masks/lhires-2400l-1785mm.mdf [spectrapy.mask.mask]
INFO: Loaded Optical Model from ./examples/tmp/lhires.opt [spectrapy.models.optmodel]


In [6]:
calib.set_trace_limits(700, 700)
calib.load_image(luci_file)
calib.plot_crv_model(7, pos=(0, 1))

In [7]:
calib.fit_crv_model()

INFO: Reloading trace positions from regions [spectrapy.modelscalib.crvcalib]
INFO: Global curvature model coeffs 0: chi2=-8.635e-33 R2=1 [spectrapy.models.crvmodel]
INFO: Global curvature model coeffs 1: chi2=-7.902e-31 R2=1 [spectrapy.models.crvmodel]
INFO: Global curvature model coeffs 2: chi2=-20.3 R2=1 [spectrapy.models.crvmodel]


In [8]:
crv.writeto("./examples/tmp/lhires.crv", overwrite=True)

inverse dispersion solution calibration

In [9]:
crv = "./examples/tmp/lhires.crv"
calib = ModelsCalibration(luci1, mask=luci_mask, opt=opt, crv=crv)

INFO: Loaded instrument configuration from ./conf/instruments/lhires-III-Halpha.icf [spectrapy.models.instrument]
INFO: Loaded mask from ./conf/masks/lhires-2400l-1785mm.mdf [spectrapy.mask.mask]
INFO: Loaded Optical Model from ./examples/tmp/lhires.opt [spectrapy.models.optmodel]
INFO: Loaded CRV model from ./examples/tmp/lhires.crv [spectrapy.models.crvmodel]


In [20]:
ids = calib.new_ids_model(1, 0, 2)

In [21]:
sky_cat = "./conf/catalogs/lhires_halpha.dat"
calib.load_image(luci_file)
calib.plot_ids_model(sky_cat, wstart=6500., nsplit = 7)

In [22]:
calib.fit_ids_model()

INFO: Global curvature model coeffs 0: chi2=0.01376 R2=0.4965 [spectrapy.models.crvmodel]
INFO: Global curvature model coeffs 1: chi2=0.02241 R2=0.9991 [spectrapy.models.crvmodel]


In [23]:
calib.plot_ids_model(sky_cat, nsplit=20)

In [24]:
ids.writeto("./examples/tmp/lhires.ids", overwrite=True)

INFO: Saved IDS model in ./examples/tmp/lhires.ids [spectrapy.models.idsmodel]


## model data tuning

curvature model data tuning

In [25]:
mask = "./conf/masks/lhires-2400l-1785mm.mdf"
inst = "./conf/instruments/lhires-III-Halpha.icf"
sc_file = "./data-reduced/gCas-360s-001.fits"

opt = "./examples/tmp/lhires.opt"
crv = "./examples/tmp/lhires.crv"

from spectrapy.datacalib.tracingcalib import TraceCalibration
trace_calib = TraceCalibration(inst, mask, opt, crv)

trace_calib.set_trace_limits(500, 1000)

trace_calib.load_image(sc_file)

trace_calib.compute_spectra_traces(slit_win=20, pix_bin=50, right=True)

INFO: Loaded instrument configuration from ./conf/instruments/lhires-III-Halpha.icf [spectrapy.models.instrument]
INFO: Loaded mask from ./conf/masks/lhires-2400l-1785mm.mdf [spectrapy.mask.mask]
INFO: Loaded Optical Model from ./examples/tmp/lhires.opt [spectrapy.models.optmodel]
INFO: Loaded CRV model from ./examples/tmp/lhires.crv [spectrapy.models.crvmodel]


In [26]:
trace_calib.fit_crv_model()

INFO: Global curvature model coeffs 0: chi2=-1.008e-31 R2=1 [spectrapy.models.crvmodel]
INFO: Global curvature model coeffs 1: chi2=-2.3e-33 R2=1 [spectrapy.models.crvmodel]
INFO: Global curvature model coeffs 2: chi2=-1.059e-32 R2=1 [spectrapy.models.crvmodel]


In [27]:
trace_calib.plot_crv_model(100, pos=(0, 0.325, 1))
trace_calib.writeto("./examples/tmp/lhires.data.crv", overwrite=True)

IDS data tuning

In [28]:
mask = "./conf/masks/lhires-2400l-1785mm.mdf"
inst = "./conf/instruments/lhires-III-Halpha.icf"
file = "./data-reduced/lamp-180s-001.fits"

crv = "./examples/tmp/lhires.data.crv"

opt = "./examples/tmp/lhires.opt"
ids = "./examples/tmp/lhires.ids"

from spectrapy.datacalib.wavelengthcalib import WavelengthCalibration
wave_calib = WavelengthCalibration(inst, mask, opt, crv, ids)
wave_calib.set_lambda_range(6480., 6650.)
wave_calib.load_image(file)
sky_cat = "./conf/catalogs/lhires_halpha.dat"

exr = wave_calib.compute_spectra_wave(sky_cat)

INFO: Loaded instrument configuration from ./conf/instruments/lhires-III-Halpha.icf [spectrapy.models.instrument]
INFO: Loaded mask from ./conf/masks/lhires-2400l-1785mm.mdf [spectrapy.mask.mask]
INFO: Loaded Optical Model from ./examples/tmp/lhires.opt [spectrapy.models.optmodel]
INFO: Loaded CRV model from ./examples/tmp/lhires.data.crv [spectrapy.models.crvmodel]
INFO: Loaded IDS model from ./examples/tmp/lhires.ids [spectrapy.models.idsmodel]
INFO: New lambda range: [6480.00, 6650.00] [spectrapy.datacalib.wavelengthcalib]
INFO: Calibrating slit test... [spectrapy.datacalib.wavelengthcalib]
INFO: Extraction table created [spectrapy.datacalib.wavelengthcalib]


In [29]:
exr.writeto("./examples/tmp/lhires.exr")

INFO: Saved extraction table: ./examples/tmp/lhires.exr [spectrapy.datacalib.extractiontable]


## spectra extraction

extraction table rectification

In [30]:
from spectrapy.datacalib.extractiontable import ExtTable
exr = ExtTable.load("./examples/tmp/lhires.exr")
exr.rectify(deg=2, margin=1)

INFO: Loaded extraction table from ./examples/tmp/lhires.exr [spectrapy.datacalib.extractiontable]
INFO: Processing slit test... [spectrapy.datacalib.extractiontable]
INFO: Done [spectrapy.datacalib.extractiontable]


spectra extraction

In [31]:
from spectrapy.extraction.exponentialfilter import ExponentialFilter
engine = ExponentialFilter(5, 1000)

In [32]:
engine.set_extraction_range(6480., 6650., 0.12)

In [45]:
from spectrapy.datacalib.extractiontable import ExtTable
#Load the extraction table and rectify it
exr=ExtTable.load('./examples/tmp/lhires.exr')

#Extract the spectra
science = "./data-reduced/saturn-180s-001.fits"
spectra2d = engine.extract(exr, science, row_start=350, row_end=480)

spectra2d.writeto("./saturn-extr.fits", overwrite=True)

INFO: Loaded extraction table from ./examples/tmp/lhires.exr [spectrapy.datacalib.extractiontable]
INFO: Extracting object test... [spectrapy.extraction.exponentialfilter]


In [43]:
from specutils import Spectrum1D

spec1d = Spectrum1D.read("./gCas-extr.fits", format="tabular-fits")

ValueError: No table found